# Analisis Data Survei "Analisis Kemudahan Akses Informasi Navigasi dalam Multikampus ITB"

## Persiapan

In [1]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import plotly.io as pio

In [9]:
detail = pd.read_csv('./data/detail.csv')
fitur = pd.read_csv('./data/fitur.csv')
masalah = pd.read_csv('./data/masalah.csv')
strings = pd.read_csv('./data/strings.csv')

## Demografi Responden


In [77]:

fig = px.pie(masalah, names='jenis_kelamin', 
             color='jenis_kelamin',
             color_discrete_map={
                 'Perempuan': '#FFB000', 
                 'Laki-laki': '#6366f1'
             })
fig.update_layout(
    title='<b>Komposisi Responden berdasarkan Jenis Kelamin</b>',
    width=800,
    height=800,
    legend=dict(font=dict(size=14))
)
fig.update_traces(textinfo='value+percent',
                  insidetextfont=dict(size=14, weight='bold'))
pio.write_image(fig, "svg/komposisi-jenis-kelamin.svg", format="svg")
fig.show()

In [72]:
df = masalah[['ID', 'mulkam', 'tingkat']].copy()
df.loc[:, 'tingkat'] = df['tingkat'].astype(str)
df.loc[:, 'tingkat'] = 'tingkat ' + df['tingkat']
fig = px.sunburst(df, 
                  path=['mulkam', 'tingkat'],
                  color='mulkam',
                  color_discrete_map={
                      'Ganesha': '#6366f1', 
                      'Jatinangor': '#FFB000'
                  })
fig.update_traces(
    textinfo='label+value',
    insidetextfont=dict(size=14, weight='bold')
)
fig.update_layout(
    title='<b>Komposisi Responden berdasarkan Asal Multikampus dan Tingkatan Akademik</b>',
    width=800,
    height=800,
    
)
pio.write_image(fig, "svg/komposisi-2.svg", format="svg")
fig.show()

In [71]:
fig = px.pie(masalah, names='disabilitas',
             color='disabilitas',
             color_discrete_map={
                 'Ya': '#6366f1', 
                 'Tidak': '#FFB000'
             })
fig.update_layout(
    title='<b>Responden dengan Disabilitas</b>',
    width=1000,
    height=1000,
    legend=dict(font=dict(size=19))
    )
fig.update_traces(textinfo='value+percent',
                  insidetextfont=dict(size=19, weight='bold'))
pio.write_image(fig, "svg/komposisi-3.svg", format="svg")
fig.show()

## Seputar Akses Informasi

In [70]:
fig = px.histogram(masalah, x="antarmulkam", 
                   color="tingkat", 
                   barmode="stack",
                   color_discrete_map={
                       1: '#6366f1',
                       2: '#DC267F',
                       3: '#FE6100',
                       4: '#FFB000',
                       5: '#648FFF'
                   },
                   category_orders={"tingkat": sorted(masalah["tingkat"].unique())}, 
                   histfunc="count")
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.update_layout(xaxis_title="Skor",
                  yaxis_title="Jumlah",
                  legend=dict(title=dict(text='<b>Tingkatan Akademik</b>')),
                  title='<b>Frekuensi Mengunjungi Multikampus Lain</b><br><span style="font-size:12px">Skala 1 (sangat jarang) - 5 (sangat sering)</span>'
                  )
pio.write_image(fig, "svg/akses-1.svg", format="svg")
fig.show()

**Insights**

Sebagian besar responden jarang berkunjung ke multikampus lain.

In [69]:
fig = px.histogram(masalah, x="kesulitan_lokasi", 
                   color="tingkat", 
                   color_discrete_map={
                       1: '#6366f1',
                       2: '#DC267F',
                       3: '#FE6100',
                       4: '#FFB000',
                       5: '#648FFF'
                   },
                   barmode="stack",
                   category_orders={"tingkat": sorted(masalah["tingkat"].unique())}, 
                   histfunc="count")
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.update_layout(xaxis_title="Skor",
                  yaxis_title="Jumlah",
                  legend=dict(title=dict(text='<b>Tingkatan Akademik</b>')),
                  title='<b>Tingkat Kesulitan Mencari Lokasi</b><br><span style="font-size:12px">Skala 1 (sangat mudah) - 5 (sangat sulit)</span>'
                  )
pio.write_image(fig, "svg/akses-2.svg", format="svg")
fig.show()

**Insights**

Mayoritas responden masih kesulitan mencari lokasi di dalam kampus. Ini mencakup mahasiswa tingkat 2, 3, dan 4 dalam persentase yang cukup signifikan (47.2%, 50%, dan 66% secara berturut-turut).

In [80]:
cross_tab = pd.crosstab(masalah['antarmulkam'], masalah['kesulitan_lokasi'])

fig = px.imshow(cross_tab,
                text_auto=True,
                title='<b>Hubungan Frekuensi Kunjungan vs Tingkat Kesulitan</b>',
                color_continuous_scale=[
                    '#6366f1',
                    '#FE6100',
                    '#FFB000',
                    ])
fig.update_layout(xaxis_title="Tingkat Kesulitan Mencari Lokasi", yaxis_title="Frekuensi Kunjungan ke Multikampus Lain")
pio.write_image(fig, "svg/akses-3.svg", format="svg")
fig.show()

**Insights**

Ada korelasi tinggi antara jarang atau sesekali mengunjungi multikampus lain (rating 2-3) dengan kesulitan mencari lokasi. Padahal, berdasarkan histogram "Frekuensi Mengunjungi Multikampus Lain", responden yang memenuhi kriteria ini mencakup lebih dari 80% total responden.

Jadi, persoalan kesulitan mencari lokasi adalah masalah yang umum dialami.

In [34]:
# bikin list of hambatan
list_hambatan = []
for hambatan in strings['Apa hambatan utama Anda saat mencari lokasi-lokasi tadi?']:
    hambatan = hambatan.split(', ')
    for entry in hambatan:
        list_hambatan.append(entry)

# itung frekuensi tiap kategori hambatan
count = pd.Series(list_hambatan).value_counts().reset_index()
count.columns = ['Hambatan', 'Jumlah']
count

fig = px.bar(count,
             x='Hambatan',
             y='Jumlah',
             title='<b>Hambatan Navigasi di Kampus</b>',
             labels={'Jumlah': '<b>Jumlah<b>', 'Hambatan': '<b>Jenis Hambatan<b>'},
             color_discrete_sequence=['#6366f1']
             )
fig.update_layout(xaxis_tickangle=-45, yaxis=dict(range=[0, count['Jumlah'].max() + 30]))
fig.update_traces(texttemplate='%{y}', textposition='inside')
pio.write_image(fig, "svg/akses-4.svg", format="svg")
fig.show()

**Insights**

Tidak ada akses denah per lantai dalam gedung menjadi penyebab kesulitan terbanyak pada navigasi dalam kampus.

In [45]:
fig = px.histogram(masalah, x="bantuan_info")
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.update_layout(xaxis_title="Skor",
                  yaxis_title="Jumlah",
                  title='<b>Efektivitas Bantuan Petugas (Satpam, OB, dsb) dalam Mencari Lokasi</b><br><span style="font-size:12px">Skala 1 (sangat kurang) - 5 (sangat cukup)</span>'
                  )
pio.write_image(fig, "svg/akses-5.svg", format="svg")
fig.show()

In [47]:
fig = px.histogram(masalah, x="setuju", color_discrete_sequence=['#6366f1'])
fig.update_traces(texttemplate='%{y}', textposition='inside')
fig.update_layout(xaxis_title="Skor",
                  yaxis_title="Jumlah",
                  title='<b>Dukungan terhadap Adanya Aplikasi Navigasi dalam Kampus</b><br><span style="font-size:12px">Skala 1 (sangat tidak setuju) - 5 (sangat setuju)</span>'
                  )
pio.write_image(fig, "svg/akses-6.svg", format="svg")
fig.show()

**Insights**

Banyak responden menganggap bantuan orang lain, khususnya petugas keamanan (satpam) maupun petugas ruangan atau petugas kebersihan sudah cukup membantu. Terlepas dari itu, hampir seluruh responden mendukung adanya aplikasi navigasi dalam kampus.

In [63]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Histogram(x=masalah['pemanfaatan_info'], name='Tingkat pemanfaatan', marker_color='#6366f1'))
fig.add_trace(go.Histogram(x=masalah['efektif_info'], name='Efektivitas', marker_color='#FFB000'))

fig.update_layout(barmode='overlay',
                  title='<b>Pemanfaatan dan Efektivitas Informasi Navigasi yang Tersedia</b><br><span style="font-size:12px">Skala 1 (sangat jarang) - 5 (sangat sering) untuk tingkat pemanfaatan, <br> Skala 1 (sangat tidak efektif) - 5 (sangat efektif) untuk efektivitas </span>',
                  xaxis_title='Skor',
                  yaxis_title='Jumlah Responden')

fig.update_traces(opacity=0.75)
fig.show()


**Insights**

Banyak responden sudah menggunakan sumber-sumber yang tersedia untuk mencari lokasi yang ingin mereka tuju di ITB. Akan tetapi, informasi ini belum cukup efektif.

## Preferensi Fitur dan Detail Aplikasi

In [61]:
df_long = fitur.melt(id_vars=['ID'],
                  var_name='Fitur',
                  value_name='Rating',
                  value_vars=["Peta interaktif", "Navigasi rute pejalan kaki", "Pencarian nama ruangan", "Mode aksesibilitas"])

fig = px.histogram(df_long,
                   x="Fitur",
                   color="Rating",
                   color_discrete_map={
                       1: '#6366f1',
                       2: '#DC267F',
                       3: '#FE6100',
                       4: '#FFB000',
                       5: '#648FFF'
                   },
                   category_orders={
                       "Rating": sorted([1, 2, 3, 4, 5]),
                       "Feature": ["Peta interaktif", "Navigasi rute pejalan kaki", "Pencarian nama ruangan", "Mode aksesibilitas"]
                   },
                   barmode='group',
                   title='<b>Prioritas Fitur Navigasi Kampus</b><br><span style="font-size:12px">Skala 1 (tidak penting) - 5 (sangat penting)</span>',
                   labels={'count': 'Jumlah Responden', 'Feature': 'Fitur'}
                   )

fig.update_layout(
    legend=dict(title=dict(text='<b>Rating</b>')),
    yaxis_title="Jumlah Vote",
    hovermode='x unified'
)


# nambahin label persentase
feature_positions = {feature: i for i, feature in enumerate(df_long['Fitur'].unique())}
rating_width = 0.16

totals = df_long.groupby('Fitur').size()
for feature in df_long['Fitur'].unique():
    for rating in [5,4,3,2,1]:
        count = len(df_long[(df_long['Fitur']==feature) & (df_long['Rating']==rating)])
        percent = count / totals[feature] * 100
        if count > 0:
            x_pos = feature_positions[feature] + (rating - 3) * rating_width
            fig.add_annotation(
                x=x_pos,
                y=count+2,
                text=f'{percent:.0f}%',
                showarrow=False,
                font=dict(size=10)
            )
pio.write_image(fig, "svg/fitur.svg", format="svg")
fig.show()

In [64]:
df_long = detail.melt(id_vars=['ID'],
                  var_name='Detail',
                  value_name='Rating',
                  value_vars=["Denah tiap lantai", "Letak toilet, kantin, dan musholla", "Area merokok", "Jalur teduh"])

fig = px.histogram(df_long,
                   x="Detail",
                   color="Rating",
                   color_discrete_map={
                       1: '#6366f1',
                       2: '#DC267F',
                       3: '#FE6100',
                       4: '#FFB000',
                       5: '#648FFF'
                   },
                   category_orders={
                       "Rating": sorted([1, 2, 3, 4, 5]),
                       "Detail": ["Denah tiap lantai", "Letak toilet, kantin, dan musholla", "Area merokok", "Jalur teduh"]
                   },
                   barmode='group',
                   title='<b>Prioritas Detail Navigasi Kampus</b><br><span style="font-size:12px">Skala 1 (tidak penting) - 5 (sangat penting)</span>',
                   labels={'count': 'Jumlah Responden', 'Feature': 'Fitur'}
                   )

fig.update_layout(
    legend=dict(title=dict(text='<b>Detail</b>')),
    yaxis_title="Jumlah Vote",
    hovermode='x unified'
)


# nambahin label persentase
detail_positions = {detil: i for i, detil in enumerate(df_long['Detail'].unique())}
rating_width = 0.16

totals = df_long.groupby('Detail').size()
for detil in df_long['Detail'].unique():
    for rating in [5,4,3,2,1]:
        count = len(df_long[(df_long['Detail']==detil) & (df_long['Rating']==rating)])
        percent = count / totals[detil] * 100
        if count > 0:
            x_pos = detail_positions[detil] + (rating - 3) * rating_width
            fig.add_annotation(
                x=x_pos,
                y=count+2,
                text=f'{percent:.0f}%',
                showarrow=False,
                font=dict(size=10)
            )
pio.write_image(fig, "svg/detail.svg", format="svg")
fig.show()